In [32]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB


This Script attempts to model the decision to either buy beer in Germany or in Denmark via stating it as a Mixed Integer Linear program problem.
Following are the variables which can be changed to the current values:

In [33]:
# Prices for soda and beer in DK and Germany as of summer 2023
p_soda_dk_avg = np.mean([90,90,80]) + 24 ## DKK
p_beer_dk_avg = np.mean([120,120,200]) + 24 ## DKK
p_soda_Ger_avg = np.mean([52,52,62]) ## DKK
p_beer_Ger_avg = np.mean([78,78,108]) ## DKK

f_van_rent = 886.4 ## DKK
d_gas_price = 14.49 ##DKK/L

KM_per_L = 14 ## KM/L

KM_DK_Ger = 338 ## KM

f_weshare = 1000 ## DKK

d_ferry_price = 319 ## DKK

d_pizzas = 240 ## DKK

van_max_capacity = 1036 ##kg
w_24_pack = 9 ##kg

max_alcohol_packs = 40 ## 24 packs

Oscar_gets_a_fine = 1000 ## DKK ## Oscar forgot his passport

Now to calculate some of the key numbers:

In [34]:
Max_packs = van_max_capacity/w_24_pack 

Gas_usage = KM_DK_Ger/KM_per_L

Gas_cost = Gas_usage*d_gas_price

print("Gas cost: ", Gas_cost)
print("Max packs: ", Max_packs)
print("Gas usage: ", Gas_usage)

#But actually we only have space for like 90 packs, until we have to start putting them on the seats
Max_packs = 90  ## 24 packs

Gas cost:  349.83
Max packs:  115.11111111111111
Gas usage:  24.142857142857142


Now to state the optimization problem


In [35]:
try:
    # Create a new model
    m = gp.Model("mip1")

    # Create variables
    x_beer_dk = m.addVar(vtype=GRB.Integer, name="x") ## 24 packs
    x_beer_Ger = m.addVar(vtype=GRB.Integer, name="x") ## 24 packs
    x_soda_dk = m.addVar(vtype=GRB.Integer, name="x") ## 24 packs
    x_soda_Ger = m.addVar(vtype=GRB.Integer, name="x") ## 24 packs
    y_dk = m.addVar(vtype=GRB.BINARY, name="y") ## Only one country
    y_Ger = m.addVar(vtype=GRB.BINARY, name="y") ## Only one country

    # Set objective
    m.setObjective((x_beer_dk*p_beer_dk_avg + x_soda_dk*p_soda_dk_avg + y_dk*f_weshare) + (x_soda_Ger*p_soda_Ger_avg + x_beer_Ger*p_beer_Ger_avg + y_ger*(d_ferry_price + d_pizzas + Gas_cost + Oscar_gets_a_fine) ), GRB.MINIMIZE)

    # Add constraint: maximum packs due to van capacity
    m.addConstr(x_beer_Ger + x_soda_Ger <= Max_packs, "c0")

    # Add constraint: maximum packs of alcohol
    m.addConstr(x_beer_Ger <= max_alcohol_packs, "c1")

    # Add constraint: only one country
    m.addConstr(y_dk + y_Ger <= 1, "c2")

    # Add constraint: Buy at least 1 beer 24 pack and 1 soda 24 pack
    m.addConstr(x_beer_Ger + x_beer_dk >= 1, "c3")

    # Add constraint: Buy at least 1 beer 24 pack and 1 soda 24 pack
    m.addConstr(x_soda_Ger + x_soda_Ger >= 1, "c4")

    # Add constraint: Can only buy beer at either DK or Ger
    m.addConstr(x_beer_Ger <= y_Ger*max_alcohol_packs, "c5")

    # Add constraint: Can only buy beer at either DK or Ger
    m.addConstr(x_soda_Ger <= y_Ger*Max_packs, "c6")

    # Add constraint: Can only buy beer at either DK or Ger
    m.addConstr(x_soda_dk + x_beer_dk <= y_dk*Max_packs, "c7")

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print('%s %g' % (v.VarName, v.X))

    print('Obj: %g' % m.ObjVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Encountered an attribute error
